<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess_sku.py -O preprocess_sku.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/model_utils.py -O model_utils.py
! unzip -o ml-1m.zip 

--2022-10-18 21:11:11--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  3.55MB/s    in 1.6s    

2022-10-18 21:11:13 (3.55 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-10-18 21:11:13--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess_sku.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6756 (6.6K) [text/plain]
Saving to: ‘preprocess_sku.py’

preprocess_sku.py   100%[===================>]

In [2]:
# ! pip install -q deepmatch

In [3]:
import pandas as pd
from ast import literal_eval
from preprocess_sku import gen_data_set, gen_model_input, gen_data_set_timesplit
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from model_utils import DNN, EmbeddingIndex, NoMask, PoolingLayer, l2_normalize,\
 reduce_mean, SampledSoftmaxLayer, get_item_embedding, sampledsoftmaxloss

In [2]:
%load_ext autoreload
%autoreload 2

In [116]:
import gc
gc.collect()

44

# Data

In [4]:
# data_path = "./"

# unames = ['user_id','gender','age','occupation','zip']
# user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
# rnames = ['user_id','sku_number','rating','timestamp']
# ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
# mnames = ['sku_number','title','genres']
# movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
# movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

# data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w
# print(data.shape)
# print(data.user_id.nunique())
# print(data.sku_number.nunique())
# data.head(2)

### Sales data

In [95]:
data = pd.read_csv('sample_data/mik_sales_180_7.csv', index_col = False, 
                   parse_dates = ['trans_date','created_time']
                   )
# data['most_recent_purchased_sku'] = data['most_recent_purchased_sku'].apply(literal_eval)
# data['most_recent_view_sku'] = data['most_recent_view_sku'].apply(literal_eval)
# data['hist_sku_number'] = data['most_recent_purchased_sku'] + data['most_recent_view_sku']
data.shape

(4321141, 10)

In [123]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2236579 entries, 84375 to 470715
Data columns (total 11 columns):
 #   Column                              Dtype         
---  ------                              -----         
 0   data_source                         object        
 1   user_id                             int64         
 2   sku_number                          int64         
 3   qty                                 int64         
 4   trans_date                          datetime64[ns]
 5   created_time                        datetime64[ns]
 6   email_address                       object        
 7   geo_zip                             int64         
 8   most_frequent_device_class_general  int64         
 9   category_path                       int64         
 10  sku_number_org                      object        
dtypes: datetime64[ns](2), int64(6), object(3)
memory usage: 204.8+ MB


In [100]:
data['user_id'].nunique(), data['sku_number'].nunique()

(1068897, 63526)

In [101]:
data['trans_date'].max(),data['trans_date'].min(), data['trans_date'].max() - data['trans_date'].min()

(Timestamp('2022-03-31 00:00:00'),
 Timestamp('2021-09-25 00:00:00'),
 Timedelta('187 days 00:00:00'))

In [102]:
import datetime
# test_start_date = pd.Timestamp(datetime.datetime.today().date() - datetime.timedelta(6))
test_start_date = data['trans_date'].max() - datetime.timedelta(6)

test_start_date

Timestamp('2022-03-25 00:00:00')

In [119]:
start_date = data['trans_date'].max() - datetime.timedelta(7) - datetime.timedelta(90)
start_date

Timestamp('2021-12-24 00:00:00')

In [122]:
data = data[data['trans_date'] >= start_date]

In [103]:
data.columns

Index(['data_source', 'user_id', 'sku_number', 'qty', 'trans_date',
       'created_time', 'email_address', 'geo_zip',
       'most_frequent_device_class_general', 'category_path'],
      dtype='object')

In [104]:
data = data[['data_source', 'user_id', 'sku_number', 'qty', 'trans_date',
       'created_time', 'email_address', 'geo_zip','most_frequent_device_class_general','category_path']]

In [105]:
data['sku_number_org'] = data['sku_number']

### Item category mapping dat

In [10]:
df_item = pd.read_csv("sample_data/mik_item_07292022.csv", index_col=False)
df_item.shape

FileNotFoundError: ignored

In [11]:
df_item.head(2)

NameError: ignored

In [ ]:
data['most_recent_view_sku'].apply(lambda x: len(x)).describe()

# Feature Engineering

In [124]:
sparse_features = ['sku_number', 'category_path',
                   'user_id', 'geo_zip','most_frequent_device_class_general']

SEQ_LEN = 50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


In [125]:
# view_item_set = list(set(data['most_recent_view_sku'].sum()))
# full_item_set = list(set(list(data['sku_number'].unique()) + view_item_set))
# len(full_item_set)

In [126]:
feature_max_idx = {}
for feature in sparse_features:
    # if feature == 'sku_number':      
    #   lbe_sku = LabelEncoder()
    #   lbe_sku.fit(full_item_set)
    #   data[feature] = lbe_sku.transform(data[feature]) + 1
    #   feature_max_idx[feature] = data[feature].max() + 1
    # else:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1 # add one to all the encoded categories labels
    feature_max_idx[feature] = data[feature].max() + 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [127]:
feature_max_idx

{'sku_number': 55185,
 'category_path': 1970,
 'user_id': 601539,
 'geo_zip': 18948,
 'most_frequent_device_class_general': 4}

In [128]:
user_profile = data[['user_id', 'geo_zip', 'most_frequent_device_class_general']].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["sku_number"]].drop_duplicates('sku_number')
user_profile.shape, item_profile.shape

((601538, 2), (55184, 1))

In [129]:
# user_item_list = data.groupby("user_id")['sku_number'].apply(list)
user_item_list = data.groupby("user_id")['sku_number'].apply(list)

user_item_list.head()

user_id
1    [9093, 49007, 2271, 49005, 1597]
2                [13262, 2085, 47698]
3                              [2766]
4               [13674, 19307, 13778]
5                             [14607]
Name: sku_number, dtype: object

In [130]:
train_set, test_set = gen_data_set_timesplit(data, test_start_date, SEQ_LEN, NUMBER_NEG_SAMPLE)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
  2%|▏         | 10213/601538 [03:21<3:14:03, 50.78it/s]


KeyboardInterrupt: ignored

In [ ]:
# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - label y
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating
# train_set[12]

In [ ]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

In [ ]:
train_model_input['user_id'].shape

# Create Embeddings

In [ ]:
from collections import Counter
train_counter = Counter(train_model_input['sku_number'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['sku_number'])]
len(train_counter), type(train_counter), len(item_count)

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

### Create Feature Embeddings



In [ ]:
# item features
item_feature_columns = ['sku_number']
# User features
user_feature_columns = ['user_id', 'geo_zip', 'most_frequent_device_class_general',                        
                        'hist_sku_number',
                        'hist_category_path',
                        'hist_len'
                        ]


In [ ]:
feature_max_idx

In [ ]:
user_sparse_feature_columns = user_feature_columns[0:3]
user_sparse_feature_columns

In [ ]:
user_seq_sparse_feature_columns = user_feature_columns[3:-1]
user_seq_sparse_feature_columns 

In [ ]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

In [ ]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns: 
   emb = Embedding(feature_max_idx[name[5:]], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=False) # True
   emb.trainable = True
   sparse_embedding[name] = emb

In [ ]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

In [ ]:
sparse_embedding # include both user and item embedding layers

# Create User Inputs


## Create Inputs Tensor

In [ ]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )
length_name = 'hist_len'
if length_name in user_feature_columns:
  user_features[length_name] = Input((1,), name=length_name, dtype='int32')

In [ ]:
user_inputs_list = list(user_features.values())
user_inputs_list

In [ ]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


In [ ]:
item_inputs_list

## Embed Inputs

In [ ]:
# Embed sparse inputs
from collections import defaultdict
user_sparse_embeded_input = defaultdict(list)

for name in user_sparse_feature_columns:
  user_sparse_embeded_input[name] = sparse_embedding[name](user_features[name])

In [ ]:
# Embed the sequence sparse inputs, need to add pooling layer to it
seq_embeded_dict = defaultdict(list)

for name in user_seq_sparse_feature_columns:
  seq_embeded_dict[name] = sparse_embedding[name](user_features[name])

# from deepctr.layers.sequence import SequencePoolingLayer
# for name in user_seq_sparse_feature_columns:
#   user_seq_sparse_embeded_input[name] = SequencePoolingLayer('mean', supports_masking=False)([seq_embeded_dict[name], user_features[length_name]])

from tensorflow.keras.layers import AveragePooling1D
user_seq_sparse_embeded_input = defaultdict(list)
for name in user_seq_sparse_feature_columns:
  user_seq_sparse_embeded_input[name] = AveragePooling1D(pool_size=50, padding='valid')(seq_embeded_dict[name])

In [ ]:
# vec = AveragePooling1D(pool_size=50, strides = 50, padding='valid')
# vec(seq_embeded_dict[name])

In [ ]:
# from deepctr.layers.sequence import SequencePoolingLayer
# vec2 = SequencePoolingLayer('mean', supports_masking=False)
# vec2([seq_embeded_dict[name], 35])

In [ ]:
# combine embeded inputs into a list
user_sparse_embedding_list = list(user_sparse_embeded_input.values()) + list(user_seq_sparse_embeded_input.values())
user_sparse_embedding_list

## Combine Embeded Input (User only)

In [ ]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

In [ ]:
user_dnn_input = Flatten()(Concatenate(axis=2)(user_sparse_embedding_list))
user_dnn_input

# Sepecify Model Layers

### User Layers

In [ ]:
from tensorflow.keras import layers

In [ ]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

In [ ]:
# from deepctr.layers import DNN
from model_utils import DNN

In [ ]:
dnn_layers = DNN(hidden_units = user_dnn_hidden_units, 
                  activation='relu', 
                  l2_reg=0, 
                  dropout_rate=0, 
                  use_bn=False, 
                  output_activation='linear',
                  seed=1024
                  )



In [ ]:
user_dnn_output = dnn_layers(user_dnn_input)
user_dnn_output = l2_normalize(user_dnn_output)
user_dnn_output

### Item Layer

In [ ]:
# Create item index tensor
item_vocabulary_size = feature_max_idx[item_feature_name]
item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

# embed the item index input
item_embeding = sparse_embedding[item_feature_name]
item_embedding_weight = NoMask()(item_embeding(item_index))
pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])
pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
pooling_item_embedding_weight


In [ ]:
pooling_item_embedding_weight.eval(session=K.get_session()).shape

In [ ]:
# .eval(session=K.get_session())

### Output Layer

In [ ]:
output = SampledSoftmaxLayer()(
    [pooling_item_embedding_weight, user_dnn_output, item_features[item_feature_name]])
output

In [ ]:
# from model_utils import SampledSoftmaxLayer
# sf = SampledSoftmaxLayer()

In [ ]:
# from tensorflow.keras.layers import Softmax
# sf2 = Softmax()
# import numpy as np
# x = np.asarray([[1., 2., 1.]])
# x
# sf2(x).eval(session=K.get_session())

In [ ]:
pooling_item_embedding_weight

In [ ]:
user_dnn_output

In [ ]:
item_features[item_feature_name]

## Build Model

In [ ]:
from tensorflow.python.keras.models import Model

In [ ]:
model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

model.__setattr__("user_input", user_inputs_list)
model.__setattr__("user_embedding", user_dnn_output)

model.__setattr__("item_input", item_inputs_list)
model.__setattr__("item_embedding",
                  get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))


# Model Compile and Training

In [ ]:
model.compile(optimizer="adam", loss=sampledsoftmaxloss)

In [ ]:
history = model.fit(train_model_input, train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

In [ ]:
res = model.predict(test_model_input)

In [ ]:
res.shape

# Predict: Generate Item and User Embeddings

In [ ]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"sku_number": item_profile['sku_number'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

In [ ]:
prob_matrix = user_embs.dot(item_embs.T)
prob_matrix.shape

# Recommendation

In [ ]:
! pip install faiss-cpu

In [ ]:
import numpy as np
import faiss
from tqdm import tqdm
# from deepmatch.utils import recall_N

In [ ]:
def recall_N(y_true, y_pred, N=50):
    return len(set(y_pred[:N]) & set(y_true)) * 1.0 / len(y_true)

In [ ]:
test_true_label = {line[0]:[line[1]] for line in test_set}

import numpy as np
import faiss
from tqdm import tqdm
# from deepmatch.utils import recall_N

index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 50)
s = []
hit = 0
pred_label = {}
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['sku_number'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
        pred_label[uid] = pred
        if test_true_label[uid] in pred:
            hit += 1
    except:
        print(i)
print("")
print("recall", np.mean(s))
print("hit rate", hit / len(test_user_model_input['user_id']))

In [ ]:
len(pred_label[160])

# Validation

In [ ]:
df_train = pd.DataFrame(train_set)

In [ ]:
test_user_model_input['user_id'][0:10]

In [ ]:
user_idx = 0
uid = 78

In [ ]:
D[user_idx]

In [ ]:
sorted_index = prob_matrix.argsort()[:,-50:]
-np.sort(-prob_matrix[user_idx][sorted_index[user_idx]])

In [ ]:
I[user_idx]

In [ ]:
(sorted_index[user_idx])[::-1]

In [ ]:
np.array([item_profile['sku_number'].values[x] for x in I[user_idx]])

In [ ]:
len(set(df_train[df_train[0]==uid][1].unique()) & set([item_profile['sku_number'].values[x] for x in I[user_idx]]))

In [ ]:
len(set(df_train[df_train[0]==uid][1].unique()) & set(sorted_index[user_idx]))

In [ ]:
item_profile['sku_number']

In [ ]:
all_item_model_input